In [ ]:
# Uncomment the following commands to install in Colab
# Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment
# and then proceed to the following cells

# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.7b2 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch finalise-notebooks
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
from aust_covid.model import build_model
import arviz as az
import numpy as np

from emutools.calibration import plot_priors, tabulate_priors, param_table_to_tex
from inputs.constants import INPUTS_PATH, SUPPLEMENT_PATH
from aust_covid.calibration import get_priors, get_all_priors, get_targets
from emutools.tex import DummyTexDoc, StandardTexDoc
from emutools.parameters import load_param_info
from aust_covid.utils import add_image_to_doc
from aust_covid.inputs import get_ifrs, get_base_vacc_data
from aust_covid.plotting import plot_targets, plot_dispersion_examples, plot_full_vacc, plot_program_coverage
from aust_covid.vaccination import get_vacc_data_masks, add_derived_data_to_vacc

if not on_colab:
    from autumn.infrastructure.remote import springboard

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Supplementary material to analysis of Australia's 2022 COVID-19 epidemic", 'austcovid')

In [ ]:
param_info = load_param_info()
all_priors = get_all_priors()
get_priors(False, param_info['abbreviations'], app_doc)
priors_table = tabulate_priors(all_priors, param_info)
app_doc.include_table(priors_table, section='Priors', col_splits=[0.25] * 4)
fig = plot_priors(all_priors, param_info['abbreviations'], 4, 100, 0.1, 0.99)
caption = 'Illustrations of prior distributions implemented in calibration algorithm.'
add_image_to_doc(fig, 'prior_distributions', caption, app_doc, 'Priors')

In [ ]:
ifrs = get_ifrs(app_doc)
param_info['value'].update(ifrs)
parameters = param_info['value'].to_dict()
prior_names = [p.name for p in get_priors(True, param_info['abbreviations'], DummyTexDoc())]
app_doc.include_table(param_table_to_tex(param_info, prior_names), section='Parameters', col_splits=[0.17, 0.15, 0.15, 0.53], longtable=True)

In [ ]:
vacc_df = get_base_vacc_data()
masks = get_vacc_data_masks(vacc_df)
vacc_df, lagged_df = add_derived_data_to_vacc(vacc_df)
full_vacc_fig = plot_full_vacc(masks['age 16+, 2+ doses'], vacc_df)
add_image_to_doc(full_vacc_fig, 'full_vacc', 'Full vaccination coverage', app_doc, 'Vaccination')
coverage_by_program = plot_program_coverage({m: masks[m] for m in ['age 16+, 3+ doses', 'age 16+, 4+ doses', 'age 12-15, 2+ doses', 'age 5-11, 2+ doses']}, vacc_df)
add_image_to_doc(coverage_by_program, 'program_coverage', 'Vaccination coverage by subsequent programs', app_doc, 'Vaccination')
vaccinated_by_modelled_group = vacc_df[['primary full', 'adult booster']].plot()
add_image_to_doc(vaccinated_by_modelled_group, 'modelled_group_vacc', 'Vaccination numbers by modelled vaccination groups', app_doc, 'Vaccination')
coverage_by_modelled_group = vacc_df[['prop primary full', 'prop adult booster']].plot()
add_image_to_doc(coverage_by_modelled_group, 'modelled_group_prop', 'Vaccination coverage by modelled vaccination groups', app_doc, 'Vaccination')
vacc_rates_by_modelled_group = vacc_df[['rate primary full', 'rate adult booster']].plot()
add_image_to_doc(vacc_rates_by_modelled_group, 'modelled_group_rates', 'Vaccination rates by modelled vaccination groups', app_doc, 'Vaccination')

In [ ]:
targets = get_targets(app_doc)
fig = plot_targets(targets)
add_image_to_doc(fig, 'target_fig', 'Calibration targets with raw data from which they were derived.', app_doc, 'Targets')

In [ ]:
epi_model = build_model(app_doc)

In [ ]:
if not on_colab:
    rts = springboard.task.RemoteTaskStore()
    rts.cd('projects/aust_covid/alternate_analyses')
    mt = rts.get_managed_task('2023-10-09T1251-none-d20k-t10k-b5k')
    # mt.download_all()
    idata = az.from_netcdf(mt.local / 'output/calib_full_out.nc')
    colours = {'notifications_ma': '10, 10, 100', 'deaths_ma': '100, 10, 10'}
    centiles = np.linspace(0.1, 0.9, 9)
    prior_names = [p.name for p in get_priors(False, param_info['abbreviations'], DummyTexDoc()) if p.name != 'imm_prop']
    fig = plot_dispersion_examples(idata, epi_model, parameters, prior_names, targets, colours, centiles)
    fig.update_layout(showlegend=False)
    caption = 'Examples of the effect of values of the negative binomial distribution dispersion parameter.'
    add_image_to_doc(fig, 'dispersion_examples', caption, app_doc, 'Targets')

In [ ]:
section_order = [
    'Model Structure',
    'Population',
    'Stratification',
    'Reinfection',
    'Mixing',
    'Initialisation',
    'Vaccination',
    'Outputs',
    'Parameters',
    'Priors',
    'Targets',
]
app_doc.write_doc(order=section_order)

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y install texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir(SUPPLEMENT_PATH)
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex supplement
    ! biber supplement
    ! pdflatex supplement
    ! pdflatex supplement